# INCOME PREDICTION FROM CENSUS DATA

Income is predicted using <b>Adult Dataset</b> which was extracted in <b>1994</b> from <b>Census Data</b> of the United States.

<b>The task is to predict whether the citizen’s income exceeds Fifty Thousand Dollars a year</b>.

It is a multivariate dataset with 14 features, out of which <b>6 are real</b> and <b>8 are nominal</b>. Features are:
<ul>
<li>Age
<li>Workclass
<li>Final Sampling Weight
<li>Education
<li>Education in Numbers
<li>Marital Status
<li>Occupation
<li>Relationship
<li>Race
<li>Gender
<li>Capital Gain
<li>Capital Loss
<li>Hours per week
<li>Country
<li/>Label
</ul>

This dataset has been obtained from the UCI Machine Learning Repository. The original page where the data set can be found is: http://archive.ics.uci.edu/ml/datasets/Adult.

# Data Preprocessing

Lets Start do some preprocessing before we start predicting using a supervised ML model.  
<ul><li>Loading Libraries
<li>Importing Data into a pandas Dataframe
<li>Skiping Header
<li>Naming Coloumns</ul>

In [1]:
import pandas as pd
import sklearn

In [2]:
df = pd.read_csv('adult.dat',skiprows=17)
df.columns = ['Age','Workclass','Final Sampling Weight','Education','Education Numbers','Marital Status','Occupation','Relationship','Race','Gender','Capital Gain','Capital Loss','Hours per week','Country','Label']
df.head()

,Age,Workclass,Final Sampling Weight,Education,Education Numbers,Marital Status,Occupation,Relationship,Race,Gender,Capital Gain,Capital Loss,Hours per week,Country,Label
0,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
1,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
2,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
3,34.0,Private,198693.0,10th,6.0,Never-married,Other-service,Not-in-family,White,Male,0.0,0.0,30.0,United-States,<=50K
4,63.0,Self-emp-not-inc,104626.0,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103.0,0.0,32.0,United-States,>50K


In [3]:
print('Dataframe has '+ str(df.shape[0]) + ' rows and ' + str(df.shape[1]) + ' coloumns')

Dataframe has 45221 rows and 15 coloumns


### Eliminating Countries other than United States

In [4]:
# since we are predicting for US only we will remove other rows
df = df[df.Country == 'United-States']

# Since all the values in Countries coloumn are same, there is no need for it. we will delete it.
df.drop('Country',axis =1,inplace=True)
df.shape

(41291, 14)

### Converting all nominal attributes to numeric type

In [5]:
df['Gender'] = df['Gender'].replace({'Male':0,'Female':1})
df['Workclass'] = df['Workclass'].replace({'Private':1,'Local-gov':2, 'Self-emp-not-inc':3, 'Federal-gov':4, 'State-gov':5,'Self-emp-inc':6, 'Without-pay':7})
df['Occupation'] = df['Occupation'].replace({'Farming-fishing':1, 'Protective-serv':2, 'Machine-op-inspct':3,'Other-service':4, 'Prof-specialty':5, 'Craft-repair':6, 'Adm-clerical':7,'Exec-managerial':8, 'Tech-support':9, 'Sales':10, 'Transport-moving':11,'Priv-house-serv':12, 'Handlers-cleaners':13, 'Armed-Forces':14})
df['Marital Status'] = df['Marital Status'].replace({'Married-civ-spouse':1, 'Never-married':2, 'Widowed':3, 'Separated':4,'Divorced':5, 'Married-spouse-absent':6, 'Married-AF-spouse':7})
df['Relationship'] = df['Relationship'].replace({'Husband':1, 'Not-in-family':2, 'Unmarried':3, 'Own-child':4, 'Wife':5,'Other-relative':6})
df['Race'] = df['Race'].replace({'White':1, 'Black':2, 'Amer-Indian-Eskimo':3, 'Asian-Pac-Islander':4,'Other':5})
df['Label'] = df['Label'].replace({'<=50K':0, '>50K':1})

# Since we already have a education in numbers coloumn we dont need education colounm of object type
df.drop('Education',axis=1,inplace=True)
df = df.astype(int,inplace=True)
df.dtypes

Age                      int64
Workclass                int64
Final Sampling Weight    int64
Education Numbers        int64
Marital Status           int64
Occupation               int64
Relationship             int64
Race                     int64
Gender                   int64
Capital Gain             int64
Capital Loss             int64
Hours per week           int64
Label                    int64
dtype: object

In [15]:
df.head()

,Age,Workclass,Final Sampling Weight,Education Numbers,Marital Status,Occupation,Relationship,Race,Gender,Capital Gain,Capital Loss,Hours per week,Label
0,38,1,89814,9,1,1,1,1,0,0,0,50,0
1,28,2,336951,12,1,2,1,1,0,0,0,40,1
2,44,1,160323,10,1,3,1,2,0,7688,0,40,1
3,34,1,198693,6,2,4,2,1,0,0,0,30,0
4,63,3,104626,15,1,5,1,1,0,3103,0,32,1


In [14]:
# class distribution
df.groupby('Label').size()

Label
0    30843
1    10448
dtype: int64

# Predictions using various Classification Models

### Patition the dataset into train (80%) and test (20%)

In [7]:
# Extracting Features from dataset 
X = df[['Age','Workclass','Final Sampling Weight','Education Numbers','Marital Status','Occupation','Relationship','Race','Gender','Capital Gain','Capital Loss','Hours per week']]
X = X.as_matrix()
print(X[0])

# Extracting labels column from dataset
Y = df[['Label']]
Y =Y.as_matrix().flatten()
print(Y)

from sklearn import model_selection
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.2, random_state=10)

print('\nLenth of Trainging set:\n',len(X_train),"\n")
print('Lenth of Test set:\n',len(X_validation))

[   38     1 89814     9     1     1     1     1     0     0     0    50]
[0 1 1 ..., 0 0 1]

Lenth of Trainging set:
 33032 

Lenth of Test set:
 8259


### Decision Tree Classifier

In [8]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(X_train, Y_train)
predictions = DT.predict(X_validation)

print('Accuracy:\n',sklearn.metrics.accuracy_score(Y_validation, predictions)*100,"\n")

Accuracy:
 81.0630826977 



### Logistic Regression 

In [9]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(X_train, Y_train)
predictions = LR.predict(X_validation)

print('Accuracy:\n',sklearn.metrics.accuracy_score(Y_validation, predictions)*100,"\n")

Accuracy:
 79.5374742705 



### Linear Discriminant Analysis

In [10]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

LDA = LinearDiscriminantAnalysis()
LDA.fit(X_train, Y_train)
predictions = LDA.predict(X_validation)

print('Accuracy:\n',sklearn.metrics.accuracy_score(Y_validation, predictions)*100,"\n")

Accuracy:
 81.910642935 



### K Neighbors

In [12]:
from sklearn.neighbors import KNeighborsClassifier

KNC = KNeighborsClassifier()
KNC.fit(X_train, Y_train)
predictions = KNC.predict(X_validation)

print('Accuracy:\n',sklearn.metrics.accuracy_score(Y_validation, predictions)*100,"\n")

Accuracy:
 77.0674415789 



### Naive Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()
GNB.fit(X_train, Y_train)
predictions = GNB.predict(X_validation)

print('Accuracy:\n',sklearn.metrics.accuracy_score(Y_validation, predictions)*100,"\n")

Accuracy:
 79.4406102434 

